In [1]:
import pandas as pd
from geopy.distance import geodesic
import datetime

In [2]:
path="../datasets/new_cleaned_data.csv"


In [3]:
df= pd.read_csv(path)
df.head()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time
0,391996,"6.508813001668548,3.37740316890347","6.650969799999999,3.3450307",2021-07-01 07:28:04,2021-07-01 07:29:37
1,391997,"6.4316714,3.4555375","6.4280814653326,3.4721885847586",2021-07-01 06:38:04,2021-07-01 07:07:28
2,391998,"6.631679399999999,3.3388976","6.508324099999999,3.3590397",2021-07-01 06:21:02,2021-07-01 07:02:23
3,391999,"6.572757200000001,3.3677082","6.584881099999999,3.3614073",2021-07-01 07:16:07,2021-07-01 07:29:42
4,392001,"6.6010417,3.2766339","6.4501069,3.3916154",2021-07-01 09:30:59,2021-07-01 09:34:36


In [76]:
df.shape

(534368, 5)

In [28]:
def calculate_distance(row):
    origin = (float(row['Trip Origin'].split(',')[0]), float(row['Trip Origin'].split(',')[1]))
    destination = (float(row['Trip Destination'].split(',')[0]), float(row['Trip Destination'].split(',')[1]))
    distance = geodesic(origin, destination).miles
    return distance
df['Distance'] = df.apply(lambda row: calculate_distance(row), axis=1)



KeyboardInterrupt: 

In [32]:
df.shape

(534368, 5)

In [60]:
def calculate_speed(row):
    distance = row['Distance']
    start_time = pd.to_datetime(row['Trip Start Time'])
    end_time = pd.to_datetime(row['Trip End Time'])
    time_diff = end_time - start_time
    if time_diff.total_seconds() == 0:
        return 0  # or some other default value you want to use
    speed = distance / time_diff.total_seconds() * 3600
    return speed
df['Speed'] = df.apply(lambda row: calculate_speed(row), axis=1)

In [81]:
print(df.columns)

Index(['Trip ID', 'Trip Origin', 'Trip Destination', 'Trip Start Time',
       'Trip End Time'],
      dtype='object')


In [4]:
import datetime

# Assuming df is your DataFrame

# Extract the dates from the Trip Start Time column
dates = [datetime.datetime.strptime(row['Trip Start Time'].split(' ')[0], '%Y-%m-%d') for index, row in df.iterrows()]

In [6]:
df['Trip_Start_Date'] = [datetime.datetime.strptime(row['Trip Start Time'].split(' ')[0], '%Y-%m-%d') for index, row in df.iterrows()]

KeyboardInterrupt: 

In [101]:
df.to_csv('date_all.csv', index=False)

In [88]:
# Create a new column 'Weekday' to store the weekday or weekend
df['Weekday'] = [1 if date.weekday() < 5 else 0 for date in dates]


In [89]:
# Save the updated DataFrame to a new CSV file
df.to_csv('dates.csv', index=False)

In [36]:
# Simplified weather classification
weather_classification = {
    "clear sky": "Sunny",
    "few clouds": "Sunny",  # Assuming few clouds still make for a generally sunny day
    "scattered clouds": "Cloudy",
    "broken clouds": "Cloudy",
    "shower rain": "Rainy",
    "rain": "Rainy",
    "thunderstorm": "Rainy",  # Thunderstorms typically involve rain
    "snow": "Rainy",         # Treating snow as rainy for simplicity, adjust as needed
    "mist": "Cloudy",        # Mist can be considered a form of cloudiness
    # Add more mappings or adjust existing ones as per your preference
}

In [9]:
import holidays
import datetime
df = pd.read_csv('date_all.csv', encoding='utf-8')


In [13]:
df.dropna(subset=['Trip Start Time'], inplace=True)

In [15]:
missingCount = df.isnull().sum()
missingCount 

Trip ID             0
Trip Origin         0
Trip Destination    0
Trip Start Time     0
Trip End Time       0
Weekday             0
Trip_Start_Date     0
dtype: int64

In [17]:

# Define the holidays for Nigeria
ng_holidays = holidays.NG()

def is_holiday(date):
    if date is None:
        return 0  # or any other default value you want
    holiday_list = ng_holidays.get(str(date))
    if holiday_list is None:
        return 0  # or any other default value you want
    return 1 if str(date) in holiday_list else 0

df['Holiday'] = df['Trip Start Time'].apply(lambda x: is_holiday(x))

In [18]:
df.to_csv('holiday.csv', index=False)

In [98]:
date = datetime.date(2023, 1, 1)  # January 1, 2023
print(is_holiday(date))  # Output: 1 if January 1 is a holiday, 0 otherwise

0


In [19]:

import requests
import datetime

# Define the OpenWeatherMap API key
api_key = "566d6148fa21834601bedfbc585af65c"

# Define the function to check if it rains on a date in Nigeria
def is_raining(date):
    # Convert the date to a string in the format YYYY-MM-DD
    date_str = date.strftime("%Y-%m-%d")

    # Fetch the weather data for the given date
    response = requests.get(f"http://api.openweathermap.org/data/2.5/weather?q=Nigeria&dt={date_str}&appid={api_key}")

    # Check if the API response is valid
    if response.status_code == 200:
        # Try to get the weather data from the response
        try:
            weather_data = response.json()
            # Check if it rains on the date
            if "rain" in weather_data.get("weather", [])[0].get("main", ""):
                return 1
            else:
                return 0
        except KeyError:
            # If the API response is not valid, return 0
            return 0
    else:
        # If the API response is not valid, return 0
        return 0

In [20]:
date = datetime.date(2023, 1, 1)
print(is_raining(date))  # Output: 1 if it rains on January 1, 2023, 0 otherwise

0


In [22]:
import datetime

# Convert the "Trip Start Time" and "Trip End Time" columns to datetime objects
start_times = []
end_times = []
for index, row in df.iterrows():
    start_times.append(datetime.datetime.strptime(row['Trip Start Time'], '%Y-%m-%d %H:%M:%S'))
    end_times.append(datetime.datetime.strptime(row['Trip End Time'], '%Y-%m-%d %H:%M:%S'))

In [23]:
# Get the unique dates from the start and end times
unique_dates = set()
for start_time in start_times:
    unique_dates.add(start_time.date())
for end_time in end_times:
    unique_dates.add(end_time.date())

In [24]:
# Convert the set of unique dates back to a list
unique_dates = list(unique_dates)
unique_dates

[datetime.date(2021, 7, 27),
 datetime.date(2021, 8, 13),
 datetime.date(2021, 9, 28),
 datetime.date(2021, 11, 5),
 datetime.date(2021, 7, 10),
 datetime.date(2022, 3, 23),
 datetime.date(2021, 10, 12),
 datetime.date(2021, 10, 1),
 datetime.date(2021, 9, 13),
 datetime.date(2022, 1, 15),
 datetime.date(2022, 1, 28),
 datetime.date(2022, 1, 14),
 datetime.date(2022, 1, 11),
 datetime.date(2021, 8, 18),
 datetime.date(2021, 7, 13),
 datetime.date(2022, 2, 23),
 datetime.date(2021, 10, 30),
 datetime.date(2021, 7, 2),
 datetime.date(2021, 8, 4),
 datetime.date(2021, 8, 16),
 datetime.date(2021, 10, 24),
 datetime.date(2021, 11, 21),
 datetime.date(2021, 9, 3),
 datetime.date(2022, 5, 19),
 datetime.date(2021, 9, 19),
 datetime.date(2021, 9, 24),
 datetime.date(2021, 8, 30),
 datetime.date(2021, 12, 12),
 datetime.date(2021, 11, 4),
 datetime.date(2021, 12, 22),
 datetime.date(2022, 4, 13),
 datetime.date(2021, 10, 7),
 datetime.date(2021, 8, 8),
 datetime.date(2021, 10, 25),
 datetime.d

In [25]:
num_unique_dates = len(unique_dates)
print(num_unique_dates)

256


In [26]:
# Create a dictionary to store the results
rain_results = {}

# Define the function to get the weather data for a given date
def get_weather_data(date):
    response = requests.get(f"http://api.openweathermap.org/data/2.5/weather?q=Nigeria&dt={date}&appid={api_key}")
    return response.json()

# Loop through the unique dates
for date in unique_dates:
    # Get the weather data for the date
    weather_data = get_weather_data(date)

    # Check if it rains on the date
    if "rain" in weather_data.get("weather", [])[0].get("main", ""):
        # Add the date to the list of raining days
        rain_results[date] = 1
    else:
        # Add the date to the list of non-raining days
        rain_results[date] = 0

# Add the "rain" column to the original DataFrame
df['rain'] = [rain_results.get(datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S').date(), False) for date in df['Trip Start Time']]

print(df)


        Trip ID                         Trip Origin  \
0        391996  6.508813001668548,3.37740316890347   
1        391997                 6.4316714,3.4555375   
2        391998         6.631679399999999,3.3388976   
3        391999         6.572757200000001,3.3677082   
4        392001                 6.6010417,3.2766339   
...         ...                                 ...   
534363  1637696         6.448218499999999,3.4772075   
534364  1637702         6.442320899999999,3.4736868   
534365  1637704                  6.4281982,3.492248   
534366  1637705                 6.5869296,3.3632966   
534367  1637709         6.647209999999999,3.4851489   

                            Trip Destination      Trip Start Time  \
0                6.650969799999999,3.3450307  2021-07-01 07:28:04   
1            6.4280814653326,3.4721885847586  2021-07-01 06:38:04   
2                6.508324099999999,3.3590397  2021-07-01 06:21:02   
3                6.584881099999999,3.3614073  2021-07-01 07:16:

In [27]:
df.to_csv('whether.csv', index=False)

In [29]:
def calculate_distance(row):
    origin = (float(row['Trip Origin'].split(',')[0]), float(row['Trip Origin'].split(',')[1]))
    destination = (float(row['Trip Destination'].split(',')[0]), float(row['Trip Destination'].split(',')[1]))
    distance = geodesic(origin, destination).miles
    return distance
df['Distance'] = df.apply(lambda row: calculate_distance(row), axis=1)

In [30]:
df.to_csv('distance.csv', index=False)

In [28]:
# Calculate the distance and speed
df['Distance'] = df.apply(lambda row: calculate_distance(row), axis=1)
df['Speed'] = df.apply(lambda row: calculate_speed(row), axis=1)

# Check if it's a holiday
df['Holiday'] = df['Trip Start Time'].apply(check_holiday)
df['Daytype'] = df['Trip Start Time'].apply(lambda x: get_weekday_or_weekend(pd.to_datetime(x)))

# Save the updated DataFrame to the CSV file
df.to_csv(path, index=False, columns=['column1', 'column2', 'column3'])

In [31]:
def calculate_speed(row):
    distance = row['Distance']
    start_time = pd.to_datetime(row['Trip Start Time'])
    end_time = pd.to_datetime(row['Trip End Time'])
    time_diff = end_time - start_time
    if time_diff.total_seconds() == 0:
        return 0  # or some other default value you want to use
    speed = distance / time_diff.total_seconds() * 3600
    return speed


In [32]:
df['Speed'] = df.apply(lambda row: calculate_speed(row), axis=1)

In [33]:
df.to_csv('speed.csv', index=False)

In [11]:
import holidays
import datetime
import pandas as pd
df = pd.read_csv('/home/miki/Logistic-optimization/driver_locations_during_request.csv', encoding='utf-8')

In [13]:
df = df.drop(['created_at', 'updated_at'], axis=1)

In [15]:
df.to_csv('request.csv', index=False)

In [14]:
missingCount = df.isnull().sum()
missingCount

id               0
order_id         0
driver_id        0
driver_action    0
lat              0
lng              0
dtype: int64

In [7]:
df.dropna(subset=['created_at'], inplace=True)
df.dropna(subset=['updated_at'], inplace=True)

In [8]:
missingCount = df.isnull().sum()
missingCount

id               0.0
order_id         0.0
driver_id        0.0
driver_action    0.0
lat              0.0
lng              0.0
created_at       0.0
updated_at       0.0
dtype: float64

In [9]:
df.to_csv('miki.csv', index=False)

In [72]:
# Assume df is your existing DataFrame
df['Distance'] = df.apply(lambda row: calculate_distance(row), axis=1)
df['Speed'] = df.apply(lambda row: calculate_speed(row), axis=1)


KeyboardInterrupt: 

In [ ]:
df['Weekday'] = df.apply(lambda row: get_weekday_or_weekend(row['Date']), axis=1)
df['Holiday'] = df.apply(lambda row: is_holiday(row['Date']), axis=1)
df['Rain'] = df.apply(lambda row: get_weather_data(row['Date']), axis=1)